<a href="https://colab.research.google.com/github/Ivan412/training/blob/main/6_python_advaced/2_import_and_parsing/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Продвинутый Python. Часть 2: Импорт и парсинг данных. Веб-скрепинг.

---


#**Задание: парсинг интернет-магазина**

Используя парсинг данных с маркетплейса beru.ru, найдите, на сколько литров отличается общий объем холодильников Саратов 263 и Саратов 452?

Для парсинга можно использовать зеркало страницы beru.ru с результатами для холодильников Саратов по адресу:

https://video.ittensive.com/data/018-python-advanced/beru.ru/

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re

In [ ]:
url = 'https://video.ittensive.com/data/018-python-advanced/beru.ru/'
goods = ['Саратов 263', 'Саратов 452']
#шаблоны поиска
goods = [re.compile(f'.*({good}).*') for good in goods]
total_vol = re.compile('общий\sобъем\s(\d{1,4})\s')

In [ ]:
def get_link(text_pattern, bs_html):
  '''
  Функция возвращает локальную ссылку на товар из родительских тэгов этого товара
  '''
  #тэг с названием товара
  tag = bs_html.find('span', string=text_pattern)
  #просматриваем родителей
  try:
    for parent in tag.parents:
      if (parent.name == 'a') and (parent.has_attr('href')):
        return parent['href']
  except:
    return ''
  return ''

In [ ]:
def get_total_volume(tvol_pattern, bs_html):
  '''
  Функция возвращает объем холодильника
  '''
  #находим тэг
  tag = bs_html.find('span', string=tvol_pattern)
  #пробуем распарсить объем и скастовать к int
  try:
    vol = tvol_pattern.search(tag.text).group(1)
    vol = int(vol)
  except:
    return 0
  return vol

In [ ]:
r = requests.get(url)
html = bs(r.content, 'lxml')
#получим ссылки на страницы с товарами
links = [url+x for x in [get_link(good, html) for good in goods] if x]

In [ ]:
#найдем объемы
r = requests.get(links[0])
html = bs(r.content, 'lxml')
s263 = get_total_volume(total_vol, html)
r = requests.get(links[1])
html = bs(r.content, 'lxml')
s462 = get_total_volume(total_vol, html)

print(f'Объем отличается на {abs(s263 - s462)} л')

Объем отличается на 73 л
